In [1]:
import numpy as np
import matplotlib.pyplot as plt

import os
import os.path
import glob

import torch, gc
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models

from torch.autograd import Variable

import torch.nn
import torch.nn.functional as F
import torch.nn.parallel

import torch.utils.data as data
from torch.utils.data import Dataset

from __future__ import print_function
import argparse
import csv

from PIL import Image


In [2]:
print(torch.cuda.is_available())
gc.collect()
torch.cuda.empty_cache()

True


In [3]:
if torch.cuda.is_available(): 
    device='cuda'
else:
    device='cpu'

# 랜덤 시드 고정
torch.manual_seed(42)

if device == 'cuda':
    torch.cuda.manual_seed_all(42)

print(torch.cuda.is_available())


True


In [4]:
batch_size = 10
training_epochs = 15
learning_rate = 0.001

In [5]:
class TrainDataset(Dataset):
    
    def __init__(self):

        self.catpaths = np.array(glob.glob('./training_set/cats/cat*.jpg'))
        self.catlabel = np.zeros(self.catpaths.shape).astype(np.long)
        self.cat = np.c_[self.catpaths, self.catlabel]

        self.dogpaths = np.array(glob.glob('./training_set/dogs/dog*.jpg'))
        self.doglabel = np.ones(self.dogpaths.shape).astype(np.long)
        self.dog = np.c_[self.dogpaths, self.doglabel]

        self.alllabel = np.r_[self.catlabel, self.doglabel]
        self.allpaths = np.r_[self.catpaths, self.dogpaths]
        self.alltrain = np.r_[self.cat, self.dog]

        self.data_transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Resize((128,128)),
            #transforms.CenterCrop((100, 100)),
            #transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

        self.data_toTensor = transforms.Compose([transforms.ToTensor()])

        print(self.alllabel)

    def __getitem__(self, index): 

        img = Image.open(self.allpaths[index])
        img = self.data_transform(img)
        label = self.alllabel[index]
        arr = np.zeros((2,))
        arr[int(label)] = 1

        return img, arr.astype(np.float32)

    def __len__(self):
        return len(self.allpaths)

class TestDataset(Dataset):
    
    def __init__(self):

        self.catpaths = np.array(glob.glob('./test_set/cats/cat*.jpg'))
        self.catlabel = np.zeros(self.catpaths.shape).astype(np.long)
        self.cat = np.c_[self.catpaths, self.catlabel]

        self.dogpaths = np.array(glob.glob('./test_set/dogs/dog*.jpg'))
        self.doglabel = np.ones(self.dogpaths.shape).astype(np.long)
        self.dog = np.c_[self.dogpaths, self.doglabel]

        self.alllabel = np.r_[self.catlabel, self.doglabel]
        self.allpaths = np.r_[self.catpaths, self.dogpaths]
        self.alltrain = np.r_[self.cat, self.dog]

        self.data_transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Resize((128,128)),
            #transforms.CenterCrop((100, 100)),
            #transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

        self.data_toTensor = transforms.Compose([transforms.ToTensor()])

        print(self.alllabel)

    def __getitem__(self, index): 

        img = Image.open(self.allpaths[index])
        img = self.data_transform(img)
        label = self.alllabel[index]
        arr = np.zeros((2,))
        arr[int(label)] = 1

        return img, arr.astype(np.float32)

    def __len__(self):
        return len(self.allpaths)




In [6]:
train_dataset = TrainDataset()
test_dataset = TestDataset()

[0 0 0 ... 1 1 1]
[0 0 0 ... 1 1 1]


In [7]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True, drop_last=True, pin_memory=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True, drop_last=True, pin_memory=True)

total_batch = len(train_loader)
test_total_batch = len(test_loader)

In [8]:
class CNN(torch.nn.Module):

    def __init__(self):
        super(CNN, self).__init__()
        self.sigmoid = torch.nn.Sigmoid()

        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        self.layer3 = torch.nn.Sequential(
            torch.nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        self.fc1 = torch.nn.Linear(16 * 16 * 128, 256, bias=True)
        self.fc2 = torch.nn.Linear(256, 64, bias=True)
        self.fc3 = torch.nn.Linear(64, 2, bias=True)

        #torch.nn.init.xavier_uniform_(self.fc1.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.fc2(out)
        out = self.fc3(out)
        out = self.sigmoid(out)
        return out

In [9]:
model = CNN().to(device)
print('총 parameter 개수 : ', sum(p.numel() for p in model.parameters() if p.requires_grad))
criterion = torch.nn.BCELoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

total_batch = len(train_loader)
print('총 배치의 수 : {}'.format(total_batch))
print(model)
print(test_total_batch)

총 parameter 개수 :  8498690
총 배치의 수 : 800
CNN(
  (sigmoid): Sigmoid()
  (layer1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=32768, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=2, bias=True)
)
202


In [10]:
lossSet=[]
testSet=[]

for epoch in range(training_epochs):
    train_avg_cost = 0
    test_avg_cost = 0

    model.train()
    for batch_idx, data in enumerate(train_loader):
        inputs, labels = data
        inputs = torch.Tensor(inputs)
        labels = torch.Tensor(labels)
        inputs, labels= inputs.cuda(), labels.cuda()
        
        
        optimizer.zero_grad()
        output = model(inputs)
        loss = criterion(output, labels)
        
        loss.backward()
        optimizer.step()
        train_avg_cost += loss/total_batch
    
    model.eval()
    with torch.no_grad():
        for batch_idx, data in enumerate(test_loader):
            inputs, labels = data
            inputs = torch.Tensor(inputs)
            labels = torch.Tensor(labels)
            inputs, labels= inputs.cuda(), labels.cuda()

            output = model(inputs)
            loss = criterion(output, labels)
        
            test_avg_cost += loss/test_total_batch
        
    print('[Epoch: {:>4}], cost = {:.9}'.format(epoch+1, train_avg_cost))  
    print('test cost : ', test_avg_cost) 
    lossSet.append(train_avg_cost.item())
    testSet.append(test_avg_cost.item())
    

[Epoch:    1], cost = 0.69056344
test cost :  tensor(0.6864, device='cuda:0')
[Epoch:    2], cost = 0.668795824
test cost :  tensor(0.6264, device='cuda:0')
[Epoch:    3], cost = 0.576286077
test cost :  tensor(0.5806, device='cuda:0')
[Epoch:    4], cost = 0.513345122
test cost :  tensor(0.5055, device='cuda:0')
[Epoch:    5], cost = 0.455990106
test cost :  tensor(0.4772, device='cuda:0')
[Epoch:    6], cost = 0.408846229
test cost :  tensor(0.4712, device='cuda:0')
[Epoch:    7], cost = 0.36798656
test cost :  tensor(0.5005, device='cuda:0')
[Epoch:    8], cost = 0.327960968
test cost :  tensor(0.4877, device='cuda:0')
[Epoch:    9], cost = 0.285151005
test cost :  tensor(0.5104, device='cuda:0')
[Epoch:   10], cost = 0.234712675
test cost :  tensor(0.5549, device='cuda:0')
[Epoch:   11], cost = 0.18978177
test cost :  tensor(0.8147, device='cuda:0')
[Epoch:   12], cost = 0.157389149
test cost :  tensor(0.8883, device='cuda:0')
[Epoch:   13], cost = 0.128797054
test cost :  tensor(1

In [12]:
%matplotlib tk
print(lossSet)
print(testSet)

#lossSet, testSet = lossSet.Torch.cpu(), testSet.Torch.cpu()
RMSEloss = np.array(list(map(np.sqrt, lossSet)))
testRMSEloss = np.array(list(map(np.sqrt, testSet)))
xdomain = np.arange(training_epochs)
xdomain += 1


ax1 = plt.subplot(121)
plt.plot(xdomain, RMSEloss, marker='o', color='red')
plt.xlabel("Epochs")
plt.ylabel("train RMSELoss")
plt.title("train RMSELoss of CNN Model")

ax2 = plt.subplot(122)
plt.plot(xdomain, testRMSEloss, marker='o', color='red')
plt.xlabel("Epochs")
plt.ylabel("test RMSELoss")
plt.title("test RMSELoss of CNN Model")

#plt.style.use(['dark_background'])
plt.tight_layout(w_pad = 4)
plt.show()

[0.690563440322876, 0.6687958240509033, 0.5762860774993896, 0.5133451223373413, 0.45599010586738586, 0.4088462293148041, 0.3679865598678589, 0.3279609680175781, 0.28515100479125977, 0.23471267521381378, 0.1897817701101303, 0.15738914906978607, 0.12879705429077148, 0.10899583995342255, 0.09405256807804108]
[0.6864036321640015, 0.6263771653175354, 0.5805774331092834, 0.5054847002029419, 0.4772105813026428, 0.47119084000587463, 0.5004812479019165, 0.487722247838974, 0.5103985667228699, 0.5548651814460754, 0.8147111535072327, 0.8883354663848877, 1.076602578163147, 1.0887190103530884, 1.2089340686798096]
